## 多任务学习（Multi-Task Learning, MTL）
<div align="center">
<img width="400" height="250" alt="image" src="https://github.com/user-attachments/assets/4dd18183-6e9e-4418-ab2b-b0f9e8edb4bb" />
</div>

多任务学习（Multi-Task Learning, MTL）是一种机器学习训练范式，核心思想是：一个模型同时学习多个相关任务，而不是像传统方法那样为每个任务单独训练模型。模型共享大部分参数，每个任务有特定输出头，联合优化多个目标。

---

# 多任务学习的数学描述

...（省略公式部分）...

## 代码示例

一个基于PyTorch的最简单Multi-Task Learning（MTL）示例，使用真实数据集（UCI Wine Quality数据集），实现两个任务：预测葡萄酒质量（回归任务）和预测葡萄酒是否优质（分类任务，质量≥6为优质）。结果将通过可视化（预测质量的散点图）和评估指标（回归的MSE、分类的准确率）来展示。


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
import pandas as pd
import matplotlib.pyplot as plt

# 定义多任务学习模型
class MultiTaskModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(MultiTaskModel, self).__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.regression_head = nn.Linear(hidden_dim, 1)
        self.classification_head = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        shared_features = self.shared(x)
        quality_pred = self.regression_head(shared_features)
        is_good_pred = self.classification_head(shared_features)
        return quality_pred, is_good_pred

# 数据准备
def prepare_data():
    data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
    X = data.drop('quality', axis=1).values
    y_quality = data['quality'].values
    y_class = (y_quality >= 6).astype(int)
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    X_train, X_test, y_quality_train, y_quality_test, y_class_train, y_class_test = train_test_split(
        X, y_quality, y_class, test_size=0.2, random_state=42
    )
    
    X_train = torch.FloatTensor(X_train)
    X_test = torch.FloatTensor(X_test)
    y_quality_train = torch.FloatTensor(y_quality_train).reshape(-1, 1)
    y_quality_test = torch.FloatTensor(y_quality_test).reshape(-1, 1)
    y_class_train = torch.FloatTensor(y_class_train).reshape(-1, 1)
    y_class_test = torch.FloatTensor(y_class_test).reshape(-1, 1)
    
    return X_train, X_test, y_quality_train, y_quality_test, y_class_train, y_class_test

# 训练模型
def train_model(model, X_train, y_quality_train, y_class_train, epochs=100, lr=0.01):
    criterion_reg = nn.MSELoss()
    criterion_cls = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        quality_pred, is_good_pred = model(X_train)
        loss_reg = criterion_reg(quality_pred, y_quality_train)
        loss_cls = criterion_cls(is_good_pred, y_class_train)
        loss = loss_reg + loss_cls
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 20 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, '
                  f'Regression Loss: {loss_reg.item():.4f}, Classification Loss: {loss_cls.item():.4f}')

# 评估和可视化
def evaluate_and_visualize(model, X_test, y_quality_test, y_class_test):
    model.eval()
    with torch.no_grad():
        quality_pred, is_good_pred = model(X_test)
        quality_pred = quality_pred.numpy()
        is_good_pred = (torch.sigmoid(is_good_pred) > 0.5).float().numpy()
        y_quality_test = y_quality_test.numpy()
        y_class_test = y_class_test.numpy()
    
    mse = mean_squared_error(y_quality_test, quality_pred)
    accuracy = accuracy_score(y_class_test, is_good_pred)
    print(f'\nTest Set Evaluation:')
    print(f'Regression MSE: {mse:.4f}')
    print(f'Classification Accuracy: {accuracy:.4f}')
    
    plt.figure(figsize=(10, 6))
    plt.scatter(y_quality_test, quality_pred, alpha=0.5)
    plt.plot([y_quality_test.min(), y_quality_test.max()], [y_quality_test.min(), y_quality_test.max()], 'r--')
    plt.xlabel('True Quality')
    plt.ylabel('Predicted Quality')
    plt.title('Wine Quality Prediction (Regression Task)')
    plt.tight_layout()
    plt.savefig('wine_quality_prediction.png')
    plt.close()
    print("Prediction scatter plot saved as 'wine_quality_prediction.png'")

    print("\nSample Predictions (First 5):")
    for i in range(5):
        print(f"Sample {i+1}: True Quality={y_quality_test[i][0]:.2f}, Predicted Quality={quality_pred[i][0]:.2f}, "
              f"True Class={y_class_test[i][0]:.0f}, Predicted Class={is_good_pred[i][0]:.0f}")

def main():
    X_train, X_test, y_quality_train, y_quality_test, y_class_train, y_class_test = prepare_data()
    model = MultiTaskModel(input_dim=11, hidden_dim=64)
    train_model(model, X_train, y_quality_train, y_class_train, epochs=100)
    evaluate_and_visualize(model, X_test, y_quality_test, y_class_test)

if __name__ == "__main__":
    main()


### 代码说明：
1. **数据集**：
   - 使用UCI Wine Quality数据集（红酒，1599条样本），包含11个化学特征和质量评分（3-8分）。
   - 任务1（回归）：预测质量分数。
   - 任务2（分类）：预测是否优质（质量≥6）。
   - 数据通过`pandas`从UCI网站加载，标准化后划分为训练集（80%）和测试集（20%）。

2. **模型结构**：
   - 共享层：两层全连接（ReLU激活），输入11维特征，隐层64维。
   - 回归头：输出1维质量分数。
   - 分类头：输出1维二分类概率（优质/非优质）。
   - 损失函数：回归用MSELoss，分类用BCEWithLogitsLoss，联合损失为两者之和。

3. **训练**：
   - 使用Adam优化器，学习率0.01，训练100个epoch。
   - 每20个epoch打印总损失、回归损失和分类损失。

4. **评估与可视化**：
   - 评估回归任务的均方误差（MSE）和分类任务的准确率。
   - 绘制散点图，展示真实质量与预测质量的关系，保存为`wine_quality_prediction.png`。
   - 打印前5个测试样本的真实和预测值（质量分数和分类结果）。

5. **依赖**：
   - 需安装`torch`、`sklearn`、`pandas`、`matplotlib`、`seaborn`（`pip install torch scikit-learn pandas matplotlib seaborn datasets`）。
   - 数据集在线加载，无需手动下载。

### 运行结果：
- 输出训练过程中的损失值。
- 测试集评估：
  - 回归任务的MSE（反映预测质量分数的误差）。
  - 分类任务的准确率（反映优质/非优质分类正确率）。
- 生成`wine_quality_prediction.png`，展示预测质量与真实质量的散点图（红线为理想预测线）。
- 打印前5个样本的预测结果，展示真实和预测的质量分数及分类结果。

### 注意：
- 热图保存在运行目录下，可用图像查看器检查。
- 模型简单（两层共享网络），适合展示MTL概念；实际应用可增加层数或使用更复杂结构。
